In [2]:
import requests
from bs4 import BeautifulSoup

# URL of the page
url = 'https://www.ledningskollen.se/Vilka-ar-med_'

# Send a GET request
response = requests.get(url)
response.raise_for_status()  # Raises an HTTPError for bad responses

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find all <li> elements with a 'data-id' attribute
lis = soup.find_all('li', attrs={'data-id': True})

# Extract data-id attributes from each <li> element
data_ids = [li['data-id'] for li in lis]

print(data_ids)


['15592597', '16906634', '11310025', '11310024', '19420087', '11310022', '15592595', '11310021', '11310019', '14419427', '11310018', '14265627', '11310017', '11310016', '13421434', '11310014', '15592594', '15592593', '15587493', '16687179', '20245744', '12003681', '18235724', '15592592', '11310011', '15592591', '15587490', '16781657', '11310008', '15592590', '17582787', '11310005', '11310004', '18522563', '18500153', '11310003', '15027793', '11310002', '11309999', '11309998', '15592589', '11309996', '11309994', '15959120', '18601689', '15592588', '16598165', '15587486', '19420086', '15735308', '16771249', '14338369', '15967782', '11309988', '11309987', '18930541', '11309986', '15592587', '11309983', '11309982', '15587483', '19703373', '20315008', '11309979', '15587482', '11309977', '19759094', '15587481', '15015054', '15587480', '15587479', '16452798', '15592585', '15959119', '19234955', '15587476', '15587475', '15592584', '15027792', '18150792', '15587485', '11309968', '15592583', '15

In [49]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_data(soup, header_class):
    """Extract data from the specified header class in the soup."""
    header = soup.find('h1' if 'Headline1' in header_class else 'h2', class_=header_class)
    ul = header.find_next('ul') if header else None
    
    return [li.text.strip() for li in ul.find_all('li')] if ul else []

def download_page(org_id):
    """Download and parse web page data for a given organization ID."""
    url = f'https://www.ledningskollen.se/Vilka-ar-med_/Visa-ledningsagare?Org={org_id}'
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        company = soup.select('h1', class_="Headline1")[0].text.strip()
        data = extract_data(soup, 'Headline1') + extract_data(soup, 'Headline2')
        return [org_id, company] + data if data else None
    except Exception as e:
        print(f"Error processing Org ID {org_id}: {e}")
        return None

def process_org_ids(org_ids):
    """Process multiple organization IDs and save the results to an Excel file."""
    results = []
    max_columns = 0
    column_names = {
        "Adress", "OrganisationsBeskrivning", "Telefon", "Hemsida",
        "Typ av ledningar", "Län", "Kommuner", "Länk till villkor för markarbeten i närheten av era ledniningar"
    }

    for org_id in org_ids:
        result = download_page(org_id)
        if result:
            # Check if the first element of data is a column name and adjust accordingly
            first_element = result[1]
            if first_element in column_names:
                # Remove the first element and add the rest under that column
                result.pop(1)
                df = pd.DataFrame([result], columns=['Org_ID'] + [f'Item_{i+1}' for i in range(len(result) - 1)])
                df[first_element] = pd.Series([result[1]], index=df.index)
            else:
                results.append(result)
                max_columns = max(max_columns, len(result))

    if results:
        # Adjusting DataFrame to accommodate varying numbers of items
        df = pd.DataFrame(results)
        df.columns = ['Org_ID'] + [f'Item_{i+1}' for i in range(max_columns - 1)]
        
        # Save the DataFrame to an Excel file
        df.to_excel('org_data.xlsx', index=False)
        print("Data has been written to org_data.xlsx")
    else:
        print("No data to write.")

# List of organization IDs to process
org_ids = [15592597]  # Update with actual IDs as needed

# Process the list of organization IDs
process_org_ids(data_ids)


Data has been written to org_data.xlsx


In [58]:
# load df from excel file
df = pd.read_excel('org_data.xlsx')

df.head()

,Org_ID,Item_1,Item_2,Item_3,Item_4,Item_5,Item_6,Item_7,Item_8,Item_9,Item_10
0,15592597,101net Käglinge samfällighetsförening,Adress101net Käglinge SamfällighetsföreningKäg...,OrganisationsbeskrivningFibernet i Käglinge/Kr...,Telefon004530515289,Hemsidahttp://www.101net.se,Typ av ledningarElektronisk kommunikation,LänSkåne län,"KommunerBurlöv, Kävlinge, Lomma, Malmö, Staffa...",NaN,NaN
1,16906634,87:ans fibernät Ek. förening,AdressGevåg 295 844 91 Hammarstrand,OrganisationsbeskrivningBya-nät för bredbandsf...,Hemsidahttps://www.87ansfiber.se/,Typ av ledningarElektronisk kommunikation,LänJämtlands län,KommunerRagunda,NaN,NaN,NaN
2,11310025,AB Bollnäs Bostäder,AdressBox 194821 22 Bollnäs,OrganisationsbeskrivningBollnäs Bostäder äger ...,Telefon0278-25900,Hemsidahttp://www.bollnasbostader.se,Typ av ledningarElektronisk kommunikation,LänGävleborgs län,KommunerBollnäs,NaN,NaN
3,11310024,AB Borlänge Energi,AdressBox 834781 28 Borlänge,"OrganisationsbeskrivningTillhandahålla el, eln...",Information om ledningsanvisninghttp://www.bor...,Hemsidawww.borlange-energi.se,"Typ av ledningarAvlopp, Dagvatten, El, Elektro...",LänDalarnas län,"KommunerBorlänge, Falun, Gagnef, Ludvika, Smed...",NaN,NaN
4,19420087,AB Göta Kanalbolag,"AdressBox 3, 52191 Motala",OrganisationsbeskrivningFörvaltar och driver G...,Telefon0141-202050,Hemsidawww.gotakanal.se,"Typ av ledningarAvlopp, El, Elektronisk kommun...","LänVästra Götalands län, Östergötlands län","KommunerKarlsborg, Linköping, Mariestad, Motal...",NaN,NaN


In [51]:
range("Organisationsbeskrivning")

TypeError: 'str' object cannot be interpreted as an integer

In [ ]:
Address, Organisationsbeskrivning, Telefon, Hemsida, Typ av ledningar, Län, Kommuner,

In [62]:
import pandas as pd

# Load the data from the Excel file
df = pd.read_excel('org_data.xlsx')

# Define the new DataFrame with the specified column names
new_columns = [
    "Adress", "Organisationsbeskrivning", "Telefon", "Hemsida",
    "Typ av ledningar", "Län", "Kommuner"
]
new_df = pd.DataFrame(columns=new_columns)

# Iterate over the rows of the original DataFrame
for index, row in df.iterrows():
    # Create a dictionary for the new row
    new_row = {col: '' for col in new_columns}
    # Iterate over each cell in the row
    for item in row:
        # Skip if cell is NaN
        if pd.isna(item):
            continue
        # Check if the cell data starts with any of the column names
        for col in new_columns:
            if str(item).startswith(col):
                # Remove the column name from the data and strip any leading/trailing whitespace
                data_without_colname = str(item).replace(col, '').strip()
                # Assign the data to the correct column in the new row
                new_row[col] = data_without_colname
                break  # Break the loop once the column is found

    # Append the processed new row to the new DataFrame
    new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)
    new_df["Företag"] = df["Item_1"]
    #Move the företag column to the first column but keep the rest of the columns in the same order
    first_column = new_df.pop('Företag')
    new_df.insert(0, 'Företag', first_column)

# Save the processed DataFrame to a new Excel file
new_df.to_excel('LK_Ledningsägare.xlsx', index=False)


In [76]:
län_dict = {}
for item in new_df["Län"].unique():
    item = item.split(", ")
    for i in item:
        län_dict.update({i: ""})

for key in län_dict.keys():
    print("\""+key+"\",")



"Skåne län",
"Jämtlands län",
"Gävleborgs län",
"Dalarnas län",
"Västra Götalands län",
"Östergötlands län",
"Norrbottens län",
"Västerbottens län",
"Gotlands län",
"Stockholms län",
"Uppsala län",
"Kronobergs län",
"Västernorrlands län",
"Blekinge län",
"Kalmar län",
"Jönköpings län",
"Hallands län",
"Västmanlands län",
"Värmlands län",
"Örebro län",
"Södermanlands län",
"",
"k till villkor för markarbeten i närheten av era ledniningarhttps://www.weum.se/installeragrava.4.2a0514cf167c609222420cda.html",


In [77]:
län_dict


{'Skåne län': '',
 'Jämtlands län': '',
 'Gävleborgs län': '',
 'Dalarnas län': '',
 'Västra Götalands län': '',
 'Östergötlands län': '',
 'Norrbottens län': '',
 'Västerbottens län': '',
 'Gotlands län': '',
 'Stockholms län': '',
 'Uppsala län': '',
 'Kronobergs län': '',
 'Västernorrlands län': '',
 'Blekinge län': '',
 'Kalmar län': '',
 'Jönköpings län': '',
 'Hallands län': '',
 'Västmanlands län': '',
 'Värmlands län': '',
 'Örebro län': '',
 'Södermanlands län': '',
 '': '',
 'k till villkor för markarbeten i närheten av era ledniningarhttps://www.weum.se/installeragrava.4.2a0514cf167c609222420cda.html': ''}

In [78]:
# for every row in new_df["Län"], if the column contains the key in län_dict, show the row
new_df[new_df["Län"].str.contains("Östergötland")]
new_df

,Företag,Adress,Organisationsbeskrivning,Telefon,Hemsida,Typ av ledningar,Län,Kommuner
0,101net Käglinge samfällighetsförening,101net Käglinge SamfällighetsföreningKäglinge ...,Fibernet i Käglinge/Kristineberg.,004530515289,http://www.101net.se,Elektronisk kommunikation,Skåne län,"Burlöv, Kävlinge, Lomma, Malmö, Staffanstorp, ..."
1,87:ans fibernät Ek. förening,Gevåg 295 844 91 Hammarstrand,Bya-nät för bredbandsfiber. Förser byarna: Krå...,,https://www.87ansfiber.se/,Elektronisk kommunikation,Jämtlands län,Ragunda
2,AB Bollnäs Bostäder,Box 194821 22 Bollnäs,Bollnäs Bostäder äger och förvaltar ca 3800 bo...,0278-25900,http://www.bollnasbostader.se,Elektronisk kommunikation,Gävleborgs län,Bollnäs
3,AB Borlänge Energi,Box 834781 28 Borlänge,"Tillhandahålla el, elnät, fjärrvärme, stadsnät...",,www.borlange-energi.se,"Avlopp, Dagvatten, El, Elektronisk kommunikati...",Dalarnas län,"Borlänge, Falun, Gagnef, Ludvika, Smedjebacken..."
4,AB Göta Kanalbolag,"Box 3, 52191 Motala",Förvaltar och driver Göta kanal,0141-202050,www.gotakanal.se,"Avlopp, El, Elektronisk kommunikation, Styrsig...","Västra Götalands län, Östergötlands län","Karlsborg, Linköping, Mariestad, Motala, Norrk..."
...,...,...,...,...,...,...,...,...
1237,"Övertorneå kommun, IT-enheten","Tingshusvägen 2,95785 Övertorneå",Kommunförvaltning,092772000,www.overtornea.se,Elektronisk kommunikation,Norrbottens län,"Haparanda, Pajala, Övertorneå"
1238,"Övertorneå kommun, VA/Gator",KommunkontorTingshusvägen 295785 Övertorneå,VA Tekniska enheten. Vatten och avloppsledning...,092772000,www.overtornea.se,"Avlopp, Dagvatten, El, Elektronisk kommunikati...",Norrbottens län,"Haparanda, Kalix, Pajala, Överkalix, Övertorneå"
1239,Övertorneå Värmeverk Aktiebolag,Energivägen 795732 Övertorneå,,092779660,,Fjärrvärme,Norrbottens län,Övertorneå
1240,Övik Energi AB,Övik Energi AB /Nät AB Hörneborgsvägen 7 89...,"Övik Energi Nät AB äger och driver elnät, Övik...",,http://www.ovikenergi.se/,"El, Elektronisk kommunikation, Fjärrkyla, Fjär...",Västernorrlands län,Örnsköldsvik


In [80]:
unique_lans = [
    "Skåne län", "Jämtlands län", "Gävleborgs län", "Dalarnas län",
    "Västra Götalands län", "Östergötlands län", "Norrbottens län",
    "Västerbottens län", "Gotlands län", "Stockholms län", "Uppsala län",
    "Kronobergs län", "Västernorrlands län", "Blekinge län", "Kalmar län",
    "Jönköpings län", "Hallands län", "Västmanlands län", "Värmlands län",
    "Örebro län", "Södermanlands län", ""
]
unique_lans.sort()
unique_lans

['',
 'Blekinge län',
 'Dalarnas län',
 'Gotlands län',
 'Gävleborgs län',
 'Hallands län',
 'Jämtlands län',
 'Jönköpings län',
 'Kalmar län',
 'Kronobergs län',
 'Norrbottens län',
 'Skåne län',
 'Stockholms län',
 'Södermanlands län',
 'Uppsala län',
 'Värmlands län',
 'Västerbottens län',
 'Västernorrlands län',
 'Västmanlands län',
 'Västra Götalands län',
 'Örebro län',
 'Östergötlands län']

## Alla bolag omsättning

In [82]:
revenue_df = pd.read_excel('LK_ledningsägare.xlsx')

In [36]:
from selenium import webdriver
from selenium.webdriver.safari.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Configure options for Safari
options = Options()
options.use_technology_preview = False  # Set to True if you want to use the Safari Technology Preview

# Initialize the Safari driver with options
driver = webdriver.Safari(options=options)

wait_time = 5
try:
    # Open the URL
    driver.get("https://www.allabolag.se/what/{company_name}")

    # Wait for the page elements to load
    wait = WebDriverWait(driver, 5)

    # Look for the cookie acceptance button by inspecting its attributes (e.g., id, class)
    # This is an example, you need to replace 'button.css-selector' with the actual selector
    try:
        cookie_button = driver.find_element(By.CLASS_NAME, 'css-iq8lad')
        cookie_button.click()
    except Exception as e:
        print("Cookie button not found or other error:", e)
    

    with open('allabolag.html', 'w') as f:
        f.write(driver.page_source)
    # Now that the pop-up is handled, continue with scraping

    link = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'h2[data-v-6628c32c]')))
    link.click()
    url = driver.current_url
    #load the page source
    print(f'current link: {url}')

    revenue = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'number--positive')))
    print(revenue.text)
finally:
    # Close the browser
    driver.quit()


current link: https://www.allabolag.se/5560781345/aktiebolaget-satila-bygg
 68 859 


In [39]:
import pandas as pd
LK_df = pd.read_excel('LK_Ledningsägare.xlsx')

In [40]:
LK_df['Företag']

0       101net Käglinge samfällighetsförening
1                87:ans fibernät Ek. förening
2                         AB Bollnäs Bostäder
3                          AB Borlänge Energi
4                          AB Göta Kanalbolag
                        ...                  
1237            Övertorneå kommun, IT-enheten
1238              Övertorneå kommun, VA/Gator
1239          Övertorneå Värmeverk Aktiebolag
1240                           Övik Energi AB
1241               Öxabäck fiber Ek. förening
Name: Företag, Length: 1242, dtype: object

In [86]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.safari.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

try:
    # Attempt to read Excel file
    df = pd.read_excel('LK_Ledningsägare.xlsx')
except FileNotFoundError:
    print("Excel file not found. Please check the file path.")
    exit(1)
except Exception as e:
    print(f"An error occurred while reading the Excel file: {e}")
    exit(1)

# Configure options for Safari
options = Options()
options.use_technology_preview = False

try:
    # Initialize the Safari driver with options
    driver = webdriver.Safari(options=options)
except Exception as e:
    print(f"Failed to initialize Safari WebDriver: {e}")
    exit(1)

# Columns for URL and Revenue
df['URL'] = ''
df['Revenue'] = ''

try:
    for index, row in df.iterrows():
        company_name = row['Företag']
        try:
            driver.get(f"https://www.allabolag.se/what/{company_name}")
            wait = WebDriverWait(driver, 3)

            # Handle the cookie button
            try:
                cookie_button = driver.find_element(By.CLASS_NAME, 'css-iq8lad')
                cookie_button.click()
            except NoSuchElementException:
                pass
            except Exception as e:
                print("Error handling cookie button for:", company_name, e)

            # Navigate and extract data
            try:
                link = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'h2[data-v-6628c32c]')))
                link.click()
                current_url = driver.current_url
                revenue = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'number--positive')))
                # Find the first h2 tag under the div with class 'company-account-figures'
                nyckeltal = driver.find_element(By.CLASS_NAME, 'company-account-figures').find_element(By.TAG_NAME, 'h2')
                
                # Update DataFrame
                df.at[index, 'Omsättning (tkr)'] = revenue.text
                df.at[index, 'Årtal'] = nyckeltal.text[:-6]
                df.at[index, 'URL allabolag.se'] = current_url
                print(f'{company_name}: {current_url}, {revenue.text}')
            except TimeoutException:
                print(f"Ingen omstättning hittades för {company_name}. Timeout while processing ")
            except Exception as e:
                print(f"Error processing {company_name}: {e}")
        except Exception as e:
            print(f"Error navigating for {company_name}: {e}")

finally:
    # Ensure WebDriver is closed properly
    driver.quit()


Ingen omstättning hittades för 101net Käglinge samfällighetsförening. Timeout while processing 
Ingen omstättning hittades för 87:ans fibernät Ek. förening. Timeout while processing 
 267 719 
AB Bollnäs Bostäder: https://www.allabolag.se/5560603176/aktiebolaget-bollnas-bostader,  267 719 
 1 189 541 
AB Borlänge Energi: https://www.allabolag.se/5560055385/aktiebolaget-borlange-energi,  1 189 541 
 59 690 
AB Göta Kanalbolag: https://www.allabolag.se/5561977587/ab-gota-kanalbolag,  59 690 
 622 884 
AB PiteEnergi: https://www.allabolag.se/5563309227/aktiebolaget-piteenergi,  622 884 
 3 392 
AB Spillings Rör: https://www.allabolag.se/5567228407/ab-spillings-ror,  3 392 
Ingen omstättning hittades för AB Storstockholms lokaltrafik. Timeout while processing 
 16 135 
AB StrömstaNET: https://www.allabolag.se/5568520802/ab-stromstanet,  16 135 
 68 859 
AB Sätila Bygg: https://www.allabolag.se/5560781345/aktiebolaget-satila-bygg,  68 859 
 221 417 
AB Tierpsbyggen: https://www.allabolag.se

In [87]:
#save the data to a new excel file
df.to_excel('LK_Ledningsägare_with_revenue.xlsx', index=False)

In [4]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.safari.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

try:
    # Attempt to read Excel file
    df = pd.read_excel('LK_Ledningsägare.xlsx')
except FileNotFoundError:
    print("Excel file not found. Please check the file path.")
    exit(1)
except Exception as e:
    print(f"An error occurred while reading the Excel file: {e}")
    exit(1)

# Configure options for Safari
options = Options()
options.use_technology_preview = False

try:
    # Initialize the Safari driver with options
    driver = webdriver.Safari(options=options)
except Exception as e:
    print(f"Failed to initialize Safari WebDriver: {e}")
    exit(1)



try:
    
    driver.get(f"https://www.allabolag.se/7696306286/87ans-fibernat-ekonomisk-forening")
    wait = WebDriverWait(driver, 4)

    # Handle the cookie button
    try:
        cookie_button = driver.find_element(By.CLASS_NAME, 'css-iq8lad')
        cookie_button.click()
    except NoSuchElementException:
        pass
    except Exception as e:
        print("Error handling cookie button for:", company_name, e)
    time.sleep(2)
    element = driver.find_element(By.XPATH, "//dt[contains(text(), 'OMSÄTTNING')]/following-sibling::dd")
    element_2 = wait.until(EC.presence_of_element_located((By.XPATH, "//dt[contains(text(), 'OMSÄTTNING')]/following-sibling::dd/following-sibling::text()")))
                
    text = element.get_attribute('textContent').strip()
    omsattning_text = element.text[:-4]
    print(omsattning_text)
    print(element_2.text[-6:-2])
                
finally:
    # Ensure WebDriver is closed properly
    driver.quit()


KeyboardInterrupt: 

In [10]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.safari.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

# Read Excel file with error handling
try:
    df = pd.read_excel('LK_Ledningsägare_with_revenue.xlsx')
except FileNotFoundError:
    print("Excel file not found. Please check the file path.")
    exit(1)
except Exception as e:
    print(f"An error occurred while reading the Excel file: {e}")
    exit(1)

# Configure Safari WebDriver
options = Options()
options.use_technology_preview = False

try:
    driver = webdriver.Safari(options=options)
except Exception as e:
    print(f"Failed to initialize Safari WebDriver: {e}")
    exit(1)

# Process each company in DataFrame
try:
    for index, row in df[1:2].iterrows():
        if pd.isna(row['Omsättning (tkr)']):
            company_name = row['Företag']
            driver.get(f"https://www.allabolag.se/what/{company_name}")
            wait = WebDriverWait(driver, 3)
            

            # Handle cookie consent button
            try:
                cookie_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'css-iq8lad')))
                cookie_button.click()
            except NoSuchElementException:
                pass
            except Exception as e:
                pass

            # Extract revenue data
            try:
                link = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'h2[data-v-6628c32c]')))
                link.click()
                current_url = driver.current_url
                #wait for element to load
                element = wait.until(EC.presence_of_element_located((By.XPATH, "//dt[contains(text(), 'OMSÄTTNING')]/following-sibling::dd")))
                element_2 = wait.until(EC.presence_of_element_located((By.XPATH, "//dt[contains(text(), 'OMSÄTTNING')]/following-sibling::dd/following-sibling::text()")))
                            
                text = element.get_attribute('textContent').strip()
                omsattning_text = element.text[:-4]

            except TimeoutException:
                
                print(f'element_info_text: {element.text}')
                print(f'element_omsättning_text: {element_2.text[-6:-2]}')
                print(f'current_url: {current_url}')
                # df.at[index, 'Omsättning (tkr)'] = omsattning_text
                # df.at[index, 'URL allabolag.se'] = current_url
                print(f"{company_name}: {omsattning_text}")
            except TimeoutException:
                print(f"Ingen omstättning hittades för {company_name}. Timeout while processing ")
            except Exception as e:
                print(f"Error processing {company_name}: {e}")
        else:
            print(f"Revenue already exists for {row['Företag']}. Skipping...")
        # df.to_excel('LK_Ledningsägare_with_revenue_2.0.xlsx', index=False)
finally:
    driver.quit()


Revenue already exists for 87:ans fibernät Ek. förening. Skipping...


In [168]:
testing_df = pd.read_excel('LK_Ledningsägare_3.0.xlsx')
testing_df['Omsättning (tkr)'].head()

for index, row in testing_df[0:160].iterrows():
    if pd.isna(row['Omsättning (tkr)']):
        
        #set the value of the row to 0
        testing_df.at[index, 'Omsättning (tkr)'] = 0
        print(f"Row {index} set to 0.")
    else:
        print(f"Row {index} has revenue data: {row['Omsättning (tkr)']}")
# save testing_df to a new excel file
testing_df.to_excel('LK_Ledningsägare_3.1.xlsx', index=False)

Row 0 set to 0.
Row 1 has revenue data: 700 - 999
Row 2 has revenue data:  267 719 
Row 3 has revenue data:  1 189 541 
Row 4 has revenue data:  59 690 
Row 5 has revenue data:  622 884 
Row 6 has revenue data:  3 392 
Row 7 set to 0.
Row 8 has revenue data:  16 135 
Row 9 has revenue data:  68 859 
Row 10 has revenue data:  221 417 
Row 11 set to 0.
Row 12 has revenue data:  95 613 
Row 13 set to 0.
Row 14 has revenue data:  1 167 568 
Row 15 has revenue data:  7 183 630 
Row 16 set to 0.
Row 17 has revenue data: 100 - 199
Row 18 has revenue data: 100 000 - 499 999
Row 19 has revenue data: 100 000 - 499 999
Row 20 has revenue data:  149 803 
Row 21 set to 0.
Row 22 has revenue data: 100 000 - 499 999
Row 23 has revenue data: 200 - 299
Row 24 has revenue data:  58 278 
Row 25 set to 0.
Row 26 has revenue data: 300 - 499
Row 27 set to 0.
Row 28 has revenue data:  47 441 
Row 29 has revenue data: 500 - 699
Row 30 has revenue data:  1 636 385 
Row 31 has revenue data: 1000 - 1499
Row 32 h

In [172]:
# how many of the rows have revenue data 
testing_df['Omsättning (tkr)'].count()

441

In [4]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.safari.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException



# Function to retrieve revenue data for a company
def get_revenue(driver, wait, company_name):
    omsattning_text = "0"
    resultat_text = ""
    text = ""
    current_url = "0"
    try:
        link = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'h2.search-results__item__title > a')))
        link.click()
        current_url = driver.current_url
    except TimeoutException:
        print("Timeout waiting for the link to be clickable.")
    except NoSuchElementException:
        print("Link not found on the page.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

    try:
        tbody = wait.until(EC.presence_of_element_located((By.TAG_NAME, 'tbody')))
        rows = tbody.find_elements(By.TAG_NAME, 'tr')

        omsattning_text = rows[0].text.split(' ')[1:]
        omsattning_text = ','.join(omsattning_text)

        resultat_text = rows[2].text.split(' ')[2:]
        resultat_text = ','.join(resultat_text)

    except Exception as e:
        print("Did not find omsättning or resultat data on the page for an AB.")

    try: 
        text = driver.find_element(By.CLASS_NAME, 'company-account-figures').find_element(By.TAG_NAME, 'h2').text[:-6]
    except Exception as e:
        print("Did not find the Årtal data on the page for an AB.")

    if omsattning_text == "" or text == "":
        try:
            element = wait.until(EC.presence_of_element_located((By.XPATH, "//dt[contains(text(), 'OMSÄTTNING')]/following-sibling::dd")))
            element_2 = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div[5]/div/div[1]/div[2]/div/div[1]/div[1]/div[2]/div[2]/dl[1]")))
                        
            omsattning_text = element.get_attribute('textContent').strip()[:-4]
            text = element_2.text[-5:-1]
            
        except Exception as e:
            print(f"Did not find any data for {company_name}")

    # If no data is found, print a message
    if text == "" and omsattning_text == "" and resultat_text == "":
        print(f"Ingen data hittades för {company_name}.")
    else:
        print("------------------------------------")
        print(f'{company_name}:\ntext: {text}')
        print(f'omsattning_text: {omsattning_text}')
        print(f'resultat_text: {resultat_text}')
        print(f'current_url: {current_url}')
        print("------------------------------------")
    return text, omsattning_text, resultat_text, current_url

# Setting up the WebDriver and options
options = Options()
options.page_load_strategy = 'normal'
driver = webdriver.Safari()
wait = WebDriverWait(driver, 1)
cookie_flag = False
# Main program execution
df = pd.read_excel('LK_Ledningsägare.xlsx')
# create the following rows if they don't exist
if 'Omsättning (tkr)' not in df.columns:
    df['Omsättning (tkr)'] = ''
if 'Resultat (tkr)' not in df.columns:
    df['Resultat (tkr)'] = ''
if 'Årtal' not in df.columns:
    df['Årtal'] = ''
if 'URL allabolag.se' not in df.columns:
    df['URL allabolag.se'] = ''
try:
    for index, row in df[178:181].iterrows():
        if pd.isna(row['Omsättning (tkr)']) or row['Omsättning (tkr)'] == '':
            company_name = row['Företag']
            driver.get(f"https://www.allabolag.se/what/{company_name}")
            if cookie_flag == False:
                try:
                    cookie_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'css-iq8lad')))
                    cookie_button.click()
                    cookie_flag = True
                except Exception as e:
                    pass
            ny_text, ny_omsattning, ny_resultat, ny_url = get_revenue(driver, wait, company_name)

            df.at[index, 'Omsättning (tkr)'] = ny_omsattning
            df.at[index, 'Resultat (tkr)'] = ny_resultat
            df.at[index, 'Årtal'] = ny_text
            df.at[index, 'URL allabolag.se'] = ny_url

            df.to_excel('LK_Ledningsägare_4.0.xlsx', index=False)
        else:
            print(f"Revenue already exists for {row['Företag']}. Skipping...")
        
finally:
    driver.quit()


Did not find omsättning or resultat data on the page for an AB.
Did not find the Årtal data on the page for an AB.
------------------------------------
Edsleskogs fibernät Ek. förening:
text: 022.
omsattning_text: 1500 - 2999
resultat_text: 
current_url: https://www.allabolag.se/what/Edsleskogs%20fibern%C3%A4t%20Ek.%20f%C3%B6rening
------------------------------------
------------------------------------
Egonet AB:
text: Nyckeltal 2023-12 
omsattning_text: 3,454,
resultat_text: 
current_url: https://www.allabolag.se/5565044814/egonet-ab
------------------------------------
Did not find omsättning or resultat data on the page for an AB.
Did not find the Årtal data on the page for an AB.
------------------------------------
Ek. förening Växjö East Fibre:
text: 022.
omsattning_text: 300 - 499
resultat_text: 
current_url: https://www.allabolag.se/7696278121/ekonomisk-forening-vaxjo-east-fibre
------------------------------------


In [36]:
driver.quit()

In [26]:
import pandas as pd
df2 = pd.read_excel('LK_Ledningsägare_4.0.xlsx')
df2['Omsättning (tkr)'].replace(' ', '', inplace=True)
# in omsättning column, in every string remove empty spaces
df2['Omsättning (tkr)'] = df2['Omsättning (tkr)'].str.replace(' ', '')
df2['Omsättning (tkr)'] = df2['Omsättning (tkr)'].str.replace(',', '')
df2['Omsättning (tkr)'] = df2['Omsättning (tkr)'].str.replace('<', '')
df2['Omsättning (tkr)'] = df2['Omsättning (tkr)'].str.replace('>', '')
# if the string contains a - sign, split the string and take the avarage of the two numbers
df2['Omsättning (tkr)'] = df2['Omsättning (tkr)'].apply(lambda x: (int(x.split('-')[0]) + int(x.split('-')[1]))/2 if '-' in x else x)
# turn the column into an integer 
df2['Omsättning (tkr)'] = df2['Omsättning (tkr)'].astype(float)
df2['Omsättning (tkr)'].value_counts()

C:\Users\kalls\AppData\Local\Temp\ipykernel_3208\1172790225.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2['Omsättning (tkr)'].replace(' ', '', inplace=True)


Omsättning (tkr)
0.0         267
299999.5     57
2249.5       56
1249.5       55
849.5        50
           ... 
48389.0       1
164644.0      1
23813.0       1
11682.0       1
904139.0      1
Name: count, Length: 397, dtype: int64

In [27]:
# save df3
df2.to_excel('LK_Ledningsägare_5.0.xlsx', index=False)